In [1898]:
import numpy as np
import matplotlib.pyplot as plt

In [1899]:
# modelling the arrival process
# per second converyor belt
# c1 uses the distribution that tells us how likely the time of the next arrival
# a random value. 

In [1900]:
# Exponential Random Number Algorithm
def poisson_est(mean):
    r=np.exp(-mean)
    N=0.0
    s=1.0
    s*=np.random.random()
    while s>r:
        N+=1
        s*=np.random.random()
    else:
        return int(N)

In [1901]:
# total number of entities conveyor1 delivers per second for 1000 seconds
c1sample=[poisson_est(1)  for i in range(10)]
np.array(c1sample).mean()
c1sample


[0, 0, 3, 2, 2, 0, 2, 1, 1, 2]

In [1902]:
# total entities conveyor belt delivers
totalc1=sum(c1sample)
totalc1

13

In [1903]:
def conveyor2(time):
    # number of entities conveyor2 delivers over a given time
    return (int(time/1.5))
conveyor2(1000)

666

In [1904]:
def failtime(frate,time):
    # per unit time
    time=time%100
    # cdf of failure 
    c_fail=1 - np.exp(-frate* time)
    if(c_fail>=np.random.random()):
        return time
    else: 
        return 0    

In [1905]:
def failnow(frate,time):
    # prob of failure in next second
    p_fail=frate*np.exp(-frate*0)+ frate*np.exp(-frate*1)

    if(p_fail>np.random.random()):
        return time
    else: 
        return 0 

In [1906]:
def fail_inverse():
    # this gives the times between events
    return (-1/0.01)*np.log(1-np.random.random())

In [1907]:
# f=np.array([failnow(0.01,time) for time in range(10000)])
f=[fail_inverse() for i in range(10000)]
f=np.array(f).astype(int)
f=np.cumsum(f)
f=f[f<=10000]


In [1908]:
81 in f
    

False

In [1909]:
# for each second, check the 
def simulate(len_time):
    C1event=[]
    stationFailures=[]
    completed=0
    downtime=0
    # stacks to count waiting entities at each conveyour belt C
    waitC1=0
    waitC2=0

# simulate events with respect to random variables
    for t in np.arange(len_time):
        C1event.append(poisson_est(1))
        stationFailures.append((-1/0.01)*np.log(1-np.random.random()))
    stationFailures=np.array(stationFailures).astype(int) 
    stationFailures=np.cumsum(stationFailures)

# simulate machine working
    # for time_index in timesteps:
    time_index=1
    while time_index<len_time:
        # if time_index>len_time:
        #     return completed

        # update parts waiting
        waitC1=C1event[time_index-1]+waitC1
        if (time_index)%1.5==0 or (time_index-0.5)%1.5==0:
            waitC2+=1

        # if it is time for failure
        if time_index in stationFailures:
            for i in range(5):
                time_index+=1
                waitC1=C1event[time_index-1]+waitC1
                if (time_index)%1.5==0 or (time_index-0.5)%1.5==0:
                    waitC2+=1
            downtime+=5
        # if both parts are ready
        if waitC1>0 and waitC2>0:
            waitC1-=1
            waitC2-=1
            completed+=1
            time_index+=1
        else:                  
            time_index+=1

    return completed, downtime, stationFailures

In [1910]:
simulate(1000)

(666,
 30,
 array([   416,    551,    586,    802,    848,    942,   1001,   1020,
          1058,   1193,   1302,   1330,   1660,   1690,   1693,   1755,
          1874,   1876,   1877,   2051,   2168,   2210,   2217,   2415,
          2430,   2491,   2531,   2659,   2712,   2989,   3026,   3102,
          3140,   3260,   3301,   3372,   3415,   3469,   3511,   3607,
          3611,   3716,   3843,   4036,   4142,   4150,   4168,   4497,
          4731,   4770,   4786,   4827,   5029,   5200,   5213,   5213,
          5217,   5323,   5750,   5785,   5816,   6280,   6308,   6379,
          6776,   6870,   6968,   6999,   7022,   7183,   7224,   7290,
          7344,   7348,   7464,   7667,   7938,   7975,   8088,   8187,
          8206,   8373,   8492,   8507,   8597,   8819,   8821,   8892,
          8981,   9454,   9479,   9491,   9506,   9870,   9908,  10099,
         10344,  10349,  10651,  10680,  10901,  11038,  11086,  11189,
         11218,  11264,  11274,  11284,  11331,  1137

In [1911]:
# will station1 do all work available instantly or one per unit time, while loop or one per iteration
# can parts + downtime> total time
# end behavior, produce or no

In [1912]:
list_of_numbers=np.array([1,2,3,4])
for i in list_of_numbers:
    list_of_numbers = np.delete(list_of_numbers,i+1)  # making the list shorter!
    print(i)

1


IndexError: index 3 is out of bounds for axis 0 with size 3